### Read Multiple Result Map

In [24]:
from absbox import API,EnginePath
localAPI = API(EnginePath.DEV,check=False)


from absbox.examples import test01,test02

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.26.4, server:0.27.10

#### What is MultiResult ?

`Multi Result` is just a map result return either by `runByScenarios()` or `runStructs()`

In [25]:
rs1 = localAPI.runStructs({"deal1":test01,"deal2":test02}
                         ,read=True)

In [26]:
p1 = ("Pool",("Mortgage",None,{"CPR":0.01},None,None)
                                ,None
                                ,None)

p2 = ("Pool",("Mortgage",None,{"CPR":0.02},None,None)
                                ,None
                                ,None)


rs2 = localAPI.runByScenarios(test01
                             ,{"CPR=1%":p1,"CPR=2%":p2}
                             ,read=True)

#### readFlowsByScenarios()

In [27]:
from absbox import readFlowsByScenarios

In [28]:
from lenses import lens

readFlowsByScenarios(rs1, lens['bonds']['A1']['balance']).head()

,deal1:balance,deal2:balance
date,,
2021-06-20,543.90,340.83
2021-07-20,429.17,224.94
2021-08-20,313.86,108.41
2021-09-20,197.87,0.00
2021-10-20,81.15,NaN


In [29]:
readFlowsByScenarios(rs1, lens['bonds']['A1']['balance'], fullName=False).head()

,balance,balance
date,,
2021-06-20,543.90,340.83
2021-07-20,429.17,224.94
2021-08-20,313.86,108.41
2021-09-20,197.87,0.00
2021-10-20,81.15,NaN


In [30]:
readFlowsByScenarios(rs2, lens['pool']['flow']['Prepayment']).head()

,CPR=1%:Prepayment,CPR=2%:Prepayment
Date,,
2021-04-01,3.47,6.98
2021-05-01,1.64,3.29
2021-06-01,1.60,3.22
2021-07-01,1.46,2.93
2021-08-01,1.42,2.85


#### readMultiFlowsByScenarios

User can have option to view `multiple columns` from a multi result. And merge them into a multi-index dataframe

In [ ]:
from absbox import readMultiFlowsByScenarios

readMultiFlowsByScenarios(rs2
                          , (lens['pool']['flow'],["Balance",'Prepayment']))

Scenario     CPR=1%              CPR=2%           
Field       Balance Prepayment  Balance Prepayment
Date                                              
2021-04-01  1989.69       3.47  1986.45       6.98
2021-05-01  1883.73       1.64  1879.09       3.29
2021-06-01  1777.20       1.60  1771.29       3.22
2021-07-01  1670.20       1.46  1663.26       2.93
2021-08-01  1562.63       1.42  1554.79       2.85
2021-09-01  1454.53       1.33  1445.99       2.66
2021-10-01  1345.93       1.20  1336.93       2.39
2021-11-01  1236.77       1.14  1227.44       2.29
2021-12-01  1127.10       1.02  1117.67       2.03
2022-01-01  1016.86       0.96  1007.49       1.91
2022-02-01   906.09       0.86   896.97       1.72
2022-03-01   794.84       0.69   786.23       1.38
2022-04-01   682.97       0.67   674.99       1.34
2022-05-01   570.55       0.56   563.43       1.11
2022-06-01   457.57       0.48   451.46       0.96
2022-07-01   344.03       0.37   339.16       0.74
2022-08-01   229.92       0.29   226.47       0.58
2022-09-01   115.25       0.19   113.42       0.38
2022-10-01     0.00       0.09     0.00       0.18